In [1]:
# Import libraries

from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib #sending emails to yourself

In [4]:
#connect to the website

URL = 'https://www.jobs.nhs.uk/candidate/jobadvert/C9188-24-0408?searchFormType=main&keyword=IT&language=en'
# Headers - https://httpbin.org/get
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

Job_title = soup2.find(id='heading').get_text()

Job_employer = soup2.find(id='employer_name').get_text()
Job_description = soup2.find(id='job_description').get_text()
Job_closing_date = soup2.find(id='closing_date').get_text()



print(Job_title)
print(Job_description)
print(Job_employer)
print(Job_closing_date)

#print(soup2)


       IT Support Analyst
      


        A rare opportunity has arisen in the Wessex Genomics Laboratory Service (WGLS) in Salisbury, to appoint a permanent full-time IT Support Analyst within our dynamic genomics IT team.
       

        WGLS is fundamentally dependent on effective and well-maintained IT resources. In liaison with the wider Trust IT department, the genomics IT team are responsible for the installation and maintenance of the IT estate within our busy, progressive genomics laboratory.
       

        As the successful candidate, you will enjoy a varied role within this team including administering a range of specialist IT systems and supporting staff with their use, problem resolution, routine service provision and maintenance tasks, and assisting with the preparation of audits for internal and external consumption. The role will be flexible in scope with potential for project or development work depending on your experience and aptitude.
       

        Applicant

In [5]:
# Clean up the data

# Trimming and slicing the job description
Job_description = Job_description.strip()[:172]

# Trimming and slicing the job closing date string and converting it to a date object
Job_closing_date = Job_closing_date.strip()[20:31]
print(Job_closing_date)

from datetime import datetime
Job_closing_date = datetime.strptime(Job_closing_date, '%d %B %Y').date()

# Trimming the employer and title
Job_employer = Job_employer.strip()
Job_title = Job_title.strip()

# Getting today's date
today = datetime.today().date()

# Printing the values
print(Job_description)
print(Job_title)
print(Job_employer)
print(Job_closing_date)

31 May 2024
A rare opportunity has arisen in the Wessex Genomics Laboratory Service (WGLS) in Salisbury, to appoint a permanent full-time IT Support Analyst within our dynamic genomics
IT Support Analyst
University Hospital Southampton NHS Trust
2024-05-31


In [ ]:
# create a csv file and add the data into it with their header
import csv

header = ['Job_title', 'Job_description','Job_employer','Job_closing_date', 'Date']
data = [Job_title, Job_description,Job_employer,Job_closing_date, today]

with open ('NHSWebScrapperDataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [ ]:
# a+ appends what ever was in the format 

with open ('NHSWebScrapperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [3]:
# Automate the whole process each day and send an Email

from bs4 import BeautifulSoup
import requests
import os
import datetime
import csv
import smtplib, ssl
import time  # Missing import

# Set your email password manually here (for testing only)
EMAIL_PASSWORD = 'xxxxxxpasswordxxxxxxx'

# Check Job function
def check_Job():
    # Connect to the web
    URL = 'https://www.jobs.nhs.uk/candidate/jobadvert/C9188-24-0408?searchFormType=main&keyword=IT&language=en'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "DNT": "1",
        "Connection": "close",
        "Upgrade-Insecure-Requests": "1"
    }
    
    # Get the whole page
    page = requests.get(URL, headers=headers)
    print("Status Code:", page.status_code)
    if page.status_code != 200:
        print("Failed to retrieve the page")
        return None
    
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    # Find and get specific data
    Job_title = soup2.find(id='heading')
    Job_employer = soup2.find(id='employer_name')
    Job_description = soup2.find(id='job_description')
    Job_closing_date = soup2.find(id='closing_date')
    
    # Ensure elements are found before trying to access their text
    if Job_title:
        Job_title = Job_title.get_text().strip()
    else:
        print("Job title not found")
        return None

    if Job_employer:
        Job_employer = Job_employer.get_text().strip()
    else:
        print("Job employer not found")
        return None

    if Job_description:
        Job_description = Job_description.get_text().strip()[0:172]
    else:
        print("Job description not found")
        return None

    if Job_closing_date:
        Job_closing_date = Job_closing_date.get_text().strip()[20:31]
    else:
        print("Job closing date not found")
        return None

    # Converting job closing date in string to date only
    from datetime import datetime
    Job_closing_date = datetime.strptime(Job_closing_date, '%d %B %Y').date()
    
    # Get today's date
    import datetime
    today = datetime.date.today()
    
    # Get the header and data to write 
    header = ['Job_title', 'Job_description', 'Job_employer', 'Job_closing_date', 'Date']
    data = [Job_title, Job_description, Job_employer, Job_closing_date, today]
    with open('NHSWebScrapperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
        
    if today < Job_closing_date:
        send_mail1()
    elif today == Job_closing_date:
        send_mail2()
        
    # Return the number of days until the job closing date
    return (Job_closing_date - today).days

# Send mail function to remind you when the closing date is same as today's date or passed it
def send_mail1():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login('testqwerty543210@gmail.com', EMAIL_PASSWORD)
    
    subject = "UPDATE!!! The NHS Job you are interested in "
    body = "Uzo, This is the moment we have been waiting for. Now is your last chance to apply for this Job at https://www.jobs.nhs.uk/candidate/jobadvert/C9163-1258E-05-24?keyword=IT&language=en . Hurry Up Uzo, Best Of Luck"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'testqwerty543210@gmail.com',
        'uzoumej@gmail.com',
        msg
    )
    server.quit()

def send_mail2():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login('testqwerty543210@gmail.com', EMAIL_PASSWORD)
    
    subject = "UPDATE!!! The NHS Job you are interested in "
    body = "Hi Uzo, The NHS Job at https://www.jobs.nhs.uk/candidate/jobadvert/C9163-1258E-05-24?keyword=IT&language=en has now expired, Please check if you got invited for the Job."
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'testqwerty543210@gmail.com',
        'uzoumej@gmail.com',
        msg
    )
    server.quit()

# Runs check_job after a set time and inputs data into your CSV
while True:
    days_until_closing = check_Job()
    if days_until_closing is None or days_until_closing >= 2:
        break
    time.sleep(86400)


Status Code: 200


In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\Macetron 03\NHSWebScrapperDataset.csv')

print(df)